In [1]:
# pip install imageio

In [7]:
import cv2
import numpy as np
import imageio

# 트랙바 이벤트 핸들러 함수
def onChange(x):
    alpha = x / 100  # 트랙바의 값(x)을 100으로 나누어 0.0 ~ 1.0 사이의 알파 값으로 변환
    # 이미지 크기를 맞추기
    img1_resized, img2_resized = resize_images(img1, img2)
    # 이미지 블렌딩
    dst = cv2.addWeighted(img1_resized, 1 - alpha, img2_resized, alpha, 0)
    cv2.imshow(win_name, dst)  # 합성된 이미지를 표시
    frames.append(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))  # 프레임을 RGB로 변환해 리스트에 저장

# 이미지 크기를 맞추는 함수
def resize_images(img1, img2):
    # 이미지 크기 맞추기
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    
    # 두 이미지 중 작은 크기에 맞추어 리사이즈
    h, w = min(h1, h2), min(w1, w2)
    img1_resized = cv2.resize(img1, (w, h))
    img2_resized = cv2.resize(img2, (w, h))
    
    return img1_resized, img2_resized

# 이미지 파일 경로 설정
img1 = cv2.imread('../img/man_face.jpg')
img2 = cv2.imread('../img/lion_face.jpg')

# 이미지가 제대로 읽혔는지 확인
if img1 is None or img2 is None:
    print("이미지 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    exit()

# 창 이름 및 트랙바 이름 설정
win_name = 'Alpha Blending'
trackbar_name = 'fade'
frames = []  # GIF로 만들 프레임을 저장할 리스트

# 이미지 표시 및 트랙바 붙이기
cv2.imshow(win_name, img1)  # 첫 번째 이미지를 먼저 창에 표시
cv2.createTrackbar(trackbar_name, win_name, 0, 100, onChange)  # 트랙바 생성
cv2.waitKey()  # 사용자가 키를 누를 때까지 대기
cv2.destroyAllWindows()  # 모든 창을 닫음

# GIF 파일로 저장
imageio.mimsave('lion-man.gif', frames, fps=10)  # 프레임을 GIF 파일로 저장


In [17]:
import cv2
import numpy as np

# --① 전경 이미지와 배경 이미지 읽기
img_fg = cv2.imread('../img/mask_hannibal.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함한 전경 이미지 읽기
img_bg = cv2.imread('../img/man_face.jpg')  # 배경 이미지 읽기

# --② 전경 이미지 크기 조정
img_fg = cv2.resize(img_fg, (348, 287))  # 전경 이미지의 크기를 배경 이미지에 맞게 조정

# --③ 알파 채널을 기준으로 마스크와 역마스크 생성
_, mask = cv2.threshold(img_fg[:,:,2], 10, 255, cv2.THRESH_BINARY)  # 전경 이미지의 알파 채널을 사용하여 마스크 생성
mask_inv = cv2.bitwise_not(mask)  # 마스크의 역마스크 생성

# --④ 전경 이미지를 배경에 삽입할 위치 계산
i, j = img_bg.shape[0] - img_fg.shape[0], (img_bg.shape[1] // 2) - (img_fg.shape[1] // 2) + 10
# i는 전경 이미지를 배경 하단에 맞추기 위한 수치
# j는 전경 이미지를 배경 중앙에 맞추기 위한 수치

# --⑤ 알파 채널 제거 및 BGR 색상으로 변환
img_fg = cv2.cvtColor(img_fg, cv2.COLOR_BGRA2BGR)  # 전경 이미지의 알파 채널을 제거하고 BGR 색상으로 변환

# --⑥ 배경 이미지에서 전경 이미지를 삽입할 영역 선택
h, w = img_fg.shape[:2]  # 전경 이미지의 높이와 너비
roi = img_bg[i:i+h, j:j+w]  # 배경 이미지에서 전경 이미지가 들어갈 영역 선택

# --⑦ 마스크를 이용하여 전경 이미지와 배경 이미지 분리
masked_fg = cv2.bitwise_and(img_fg, img_fg, mask=mask)  # 전경 이미지에서 마스크에 해당하는 부분만 남김
masked_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)  # 배경 이미지에서 마스크에 해당하지 않는 부분만 남김

# --⑧ 전경 이미지와 배경 이미지 합성
added = masked_fg + masked_bg  # 전경 이미지와 배경 이미지를 합성

# --⑨ 결과 이미지에 전경 이미지를 삽입
img_bg[i:i+h, j:j+w] = added  # 최종 이미지를 배경 이미지에 삽입

# --⑩ 결과 출력
cv2.imshow('mask', mask)        # 마스크 이미지 출력
cv2.imshow('mask_inv', mask_inv)  # 역마스크 이미지 출력
cv2.imshow('masked_fg', masked_fg)  # 마스크가 적용된 전경 이미지 출력
cv2.imshow('masked_bg', masked_bg)  # 마스크가 적용된 배경 이미지 출력
cv2.imshow('added', added)      # 전경과 배경이 합성된 이미지 출력
cv2.imshow('result', img_bg)   # 최종 결과 이미지 출력 (전경이 배경에 삽입된 상태)

cv2.waitKey()  # 사용자가 키를 누를 때까지 대기
cv2.destroyAllWindows()  # 모든 창을 닫음
